<a href="https://colab.research.google.com/github/gatherheart/Classification_With_BERT/blob/main/news_classifications_pytorch_kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 55.0MB 103kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 8.9MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588512 sha256=59b65a851accd2442cb00486080a3d66df46c5dc98dc760ef18809203ff036da
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.1MB 9.1MB/s 
     |████████████████████████████████| 757kB 8.0MB/s 
     |████████████████████████████████| 3.0MB 16.9MB/s 
     |████████████████████████████████| 890kB 61.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a5a3db2530edb489fd5ed765bbb60777179d198cea6b34262b24642d135e9b9a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a501

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-rdr2tnsv
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-rdr2tnsv
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=47162fd1582ab7164b5846177428cb288c8fe3b60575b8be0de53fd58d552044
  Stored in directory: /tmp/pip-ephem-wheel-cache-px6deap3/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook, notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
import tensorflow as tf

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [8]:
##GPU 사용 시
device = torch.device("cuda:0")

In [9]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [10]:
BASE_PATH = '/content/drive/MyDrive/hyun_data'

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Dec 12 04:40:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
df = pd.read_csv(BASE_PATH+'/preprocessed_norm.csv', index_col=[0])

In [12]:
labels_csv = pd.read_csv(BASE_PATH+'/labels_dict2.csv', index_col=[0])

In [13]:
labels_csv

,문화>방송_연예,국제>중국,사회>사회일반,정치>국회_정당,사회>교육_시험,IT_과학>모바일,정치>행정_자치,사회>사건_사고,문화>문화일반,정치>청와대,사회>여성,국제>유럽_EU,경제>자원,국제>미국_북미,사회>날씨,사회>의료_건강,IT_과학>콘텐츠,경제>증권_증시,정치>선거,국제>일본,사회>미디어,문화>음악,국제>중남미,국제>중동_아프리카,경제>유통,문화>출판,경제>자동차,사회>환경,경제>금융_재테크,사회>장애인,문화>전시_공연,경제>부동산,IT_과학>IT_과학일반,문화>요리_여행,IT_과학>과학,경제>서비스_쇼핑,경제>외환,미분류,문화>미술_건축,문화>생활,경제>경제일반,국제>아시아,정치>정치일반,경제>국제경제,경제>취업_창업,IT_과학>보안,정치>외교,경제>무역,문화>영화,정치>북한,경제>산업_기업,문화>종교,경제>반도체,문화>학술_문화재,사회>노동_복지,국제>국제일반,IT_과학>인터넷_SNS,국제>러시아
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57


In [14]:
df.columns

Index(['date', 'title', 'topic', 'keyword', 'input', 'body', 'label'], dtype='object')

In [15]:
df[df.label==0]

,date,title,topic,keyword,input,body,label
8,20201128,"'온앤오프' 이지아, '데뷔 후 최초' 베일 싸여있던 일상 공개",문화>방송_연예,"온앤오프,이지아,데뷔,베일,일상,공개,28일,방송,tvN,온앤오프,배우,이지아,반전...",이지아 온앤오프 덕후 한경닷컴 김나경 tvn 은하계 ufo 동작 하나하나 외계인설 ...,28일(토) 방송하는 tvN ’온앤오프‘에서는 배우 이지아의 반전 매력 넘치는 일상...,0
10,20201128,"'런닝맨' 김소연, '팬심' 폭발시킨 멤버 정체는?",문화>방송_연예,"런닝맨,김소연,팬심,폭발,멤버,정체,배우,김소연,팬심,고백,29일,방송,SBS,런닝...",김소연 런닝맨 팬심 이상우 멤버들 리액션 반박해 180도 다들 거짓말 펜트하우스 1...,배우 김소연이 수줍은 팬심을 고백한다.\n오는 29일 오후 방송되는 SBS ‘런닝맨...,0
15,20201128,"‘트로트의 민족’ 김소연, 우승 후보→패자부활전 합격할까[M+TV인사이드]",문화>방송_연예,"트로트,민족,김소연,우승,후보,패자부활전,합격,TV인사이드,트로트,민족,유력,우승,...",트로트 박하명 긴장감 김소연 안성준 시청률 신명근 생존자 패자부활전 장명서 나미애 ...,‘트로트의 민족’ 유력 우승 후보 김소연이 패자부활전에 나선 가운데 그의 생존에 관...,0
21,20201128,"'아는 형님' 조병규, 즉석 액션부터 성대모사까지 매력 대방출!",문화>방송_연예,"형님,조병규,즉석,액션,성대모사,매력,대방출,배우,조병규,예능,블루칩,등극,방송,J...",조병규 김세정 눈물 성대모사 유준상 친밀도 만큼 촬영장 웃픈 블루칩 자유자재 전학생...,배우 조병규가 넘치는 끼로 예능 블루칩에 등극했다.\n \n28일 방송되는 JTBC...,0
67,20201128,‘온앤오프’ 이지아 “UFO 출몰 지역을 간 적도 있다”,문화>방송_연예,"온앤오프,이지아,UFO,출몰,지역,28일,방송,tvN,온앤오프,배우,이지아,반전,매...",이지아 온앤오프 덕후 ufo tvn 동작 하나하나 은하계 외계인설 싱어송라이터 놀라...,[헤럴드경제 = 서병기 선임기자]28일(토) 밤 방송되는 tvN ’온앤오프‘에서는 ...,0
...,...,...,...,...,...,...,...
19069,20201201,영화보며 시간 여행 자연 힐링 독서하며 언택트 송년,문화>방송_연예,"영화,여행,자연,힐링,독서,언택트,송년,연말,영화,리틀,포레스트,Little,개봉,...",구멍가게 일본 순천만 미국 곽재구 루나나 해남 목포 원스 혜원 보이저 시간여행 드루...,[연말에 보면 좋은 영화]\n▲리틀 포레스트(Little Forest /감독 임순례...,0
19304,20201201,"[종합] 아이랜드 통해 데뷔한 '엔하이픈' ""7명 전원이 센터급 비주얼"" 연말 신...",문화>방송_연예,"아이랜드,데뷔,엔하이픈,전원,센터,비주얼,연말,신인상,사진,빌리프랩,제공,빅히트엔터...",엔하이픈 아이랜드 빅히트 니키 희승 제이크 제이 타이틀곡 성훈 7명 신인상 팬들 방...,[사진= 빌리프랩 제공] \n \n \n 빅히트엔터테인먼트와 Mnet의 프로젝트 ...,0
19581,20201130,"[이웃사랑] 뇌병변에 유방암까지 온 아내, 홀로 고군분투 중인 남편",문화>방송_연예,"뇌병변,유방암,아내,고군분투,남편,결혼식,뇌병변,장애,남편,4년,병간호,유방암,생활...",주희 영석 결혼식 예주 유방암 중국집 대구은행 병간호 핸드폰 일자리 생활비 매일신문...,지난 27일 달서구 감삼동의 한 주택 2층. 쿵쾅쿵쾅 계단 위로 한 여자아이가 급히...,0
19598,20201130,전효성 다이어트 “3개월만에 허리 26인치→23.5인치”,문화>방송_연예,"3개월,전효,다이어트,허리,인치,23.5인치,가수,전효,다이어트,영상,공개,화제,전...",전효성 전효 3개월 다이어터 필라테스 배달음식 필요성 유튜브 장장 2개월 식이요법 ...,가수 전효성이 다이어트 영상을 공개해 화제다.\n\n\n\n전효성이 30일 오후 자...,0


In [16]:
NUM_LABELS = max(set(df['label'])) + 1
NUM_LABELS

58

In [17]:
text_to_label = dict(labels_csv)

In [18]:
label_to_text = {}
for label in text_to_label:
    label_to_text[int(text_to_label[label])] = label

In [19]:
label_to_text

{0: '문화>방송_연예',
 1: '국제>중국',
 2: '사회>사회일반',
 3: '정치>국회_정당',
 4: '사회>교육_시험',
 5: 'IT_과학>모바일',
 6: '정치>행정_자치',
 7: '사회>사건_사고',
 8: '문화>문화일반',
 9: '정치>청와대',
 10: '사회>여성',
 11: '국제>유럽_EU',
 12: '경제>자원',
 13: '국제>미국_북미',
 14: '사회>날씨',
 15: '사회>의료_건강',
 16: 'IT_과학>콘텐츠',
 17: '경제>증권_증시',
 18: '정치>선거',
 19: '국제>일본',
 20: '사회>미디어',
 21: '문화>음악',
 22: '국제>중남미',
 23: '국제>중동_아프리카',
 24: '경제>유통',
 25: '문화>출판',
 26: '경제>자동차',
 27: '사회>환경',
 28: '경제>금융_재테크',
 29: '사회>장애인',
 30: '문화>전시_공연',
 31: '경제>부동산',
 32: 'IT_과학>IT_과학일반',
 33: '문화>요리_여행',
 34: 'IT_과학>과학',
 35: '경제>서비스_쇼핑',
 36: '경제>외환',
 37: '미분류',
 38: '문화>미술_건축',
 39: '문화>생활',
 40: '경제>경제일반',
 41: '국제>아시아',
 42: '정치>정치일반',
 43: '경제>국제경제',
 44: '경제>취업_창업',
 45: 'IT_과학>보안',
 46: '정치>외교',
 47: '경제>무역',
 48: '문화>영화',
 49: '정치>북한',
 50: '경제>산업_기업',
 51: '문화>종교',
 52: '경제>반도체',
 53: '문화>학술_문화재',
 54: '사회>노동_복지',
 55: '국제>국제일반',
 56: 'IT_과학>인터넷_SNS',
 57: '국제>러시아'}

In [20]:
train, test = train, test = train_test_split(df, test_size=0.2)

print(train.shape)
print(test.shape)

(38783, 7)
(9696, 7)


In [21]:
train.head(10)

,date,title,topic,keyword,input,body,label
2201,20201204,"권덕철, 복지부 관료로 ‘잔뼈’ 정영애, 노무현 청와대 비서관",정치>청와대,"권덕철,복지부,관료,잔뼈,정영애,비서관,노무현,청와대,내정자들,신임,장관,내정자,행...",노무현 비서관 복지부 청와대 내정자 행정학 권덕철 검경 더불어민주당 전해철 여성학 ...,ㆍ신임 장관 내정자들은 누구\n\n행정안전부 장관에 낙점된 전해철 더불어민주당 의원...,9
4541,20201205,‘秋 尹 갈등’이 기름 부은 ‘정권 교체론’,정치>선거,"갈등,기름,정권,교체,선거,대통령,야당,후보,당선,응답,여당,후보,당선,응답,3%포...",지지율 한국갤럽 법무부 갤럽 홈페이지 윤석열 임대차 민주당 18세 우상호 더불어민주...,[헤럴드경제=김용재 기자]다음 대통령 선거에서 ‘야당 후보가 당선되어야 한다’는 응...,18
15374,20201201,수출로 날았다 코로나19에도 3분기 경제성장률 반등 배경,경제>무역,"수출,코로나19,경제,성장,반등,배경,코로나19,확산,경제,성장,2.1%,11년,최...",코로나19 한은 성장률 반도체 건설업 1인 경제성장률 3만 설비투자 한국은행,코로나19의 2차 확산에도 3분기 경제성장률(2.1%)이 11년 만에 최고치를 기록...,47
17000,20201126,"종근당 이장한 회장 리더십으로 일군 국민유산균`락토핏`, 올해 2600억원 매출 신...",경제>산업_기업,"종근당,이장,회장,리더십,국민유산균,락토핏,2600억,예상,매출,시장,프로바이오틱스...",락토핏 유산균 소비자 종근당 프로바이오틱스 1위 종근당건강 성장세 계열사 건강기능식...,국내 프로바이오틱스 시장이 전년도에 이어 폭발적 성장세를 이어가고 있는 가운데 종근...,50
9085,20201203,"영국, 화이자 백신 긴급 승인 90세 여왕 총리도 맞을까",국제>유럽_EU,"영국,백신,긴급,90세,여왕,총리,영국,미국,화이자,감염증,신종,코로나바이러스,코로...",영국 코로나19 존슨 요양원 만큼 노동당 보건장관 버킹엄궁 80세 엘리자베스 90세...,영국이 미국 화이자가 개발한 신종 코로나바이러스 감염증(코로나19) 백신을 승인하면...,11
13874,20201120,주택 투자 막으니 틈새상품 '생활형 숙박시설' 인기,경제>부동산,"주택,투자,숙박시설,틈새,상품,생활,숙박,시설,인기,투시도,오션,스테이,양양,부동산...",부동산 생활형 평촌 양양 다주택자 숙박시설 1만 취득세 투자처 풀퍼니시드 건축법 양...,오션스테이 양양 투시도 \n \n \n부동산 투자시장에서 이른바 틈새상품들이 인기...,31
17669,20201201,삼성전자 SK하이닉스 덕분에 3분기 2%대 성장률 달성,경제>유통,"삼성전자,SK하이닉스,덕분,3분기,달성,성장,수출회복,가시화,설비투자,회복,경제,성...",속보치 반도체 3만 설비투자 1인 순수출 gdp 경제통계부장 증가율 국민소득 첨단공...,"[머니투데이 한고은 기자, 고석용 기자] \n \n수출회복이 가시화되고, 설비투자가...",24
11081,20201120,'애로부부' 미국 변호사 서동주 '안성맞춤 게스트'로 출연,문화>방송_연예,"애로부부,게스트,미국,변호사,서동주,안성맞춤,출연,토크쇼,부부,토크,애로부부,애로부...",서동주 애로부부 변호사 미국 방송인 최화정 양재진 법조인 안성맞춤 토크쇼 이상아 홍...,19금 부부 토크쇼 ‘다시 뜨거워지고 싶은 애로부부(이하 애로부부)’에 미국 변호사...,0
2629,20201204,"국제사이버대학교, 2021학년도 신ㆍ편입생 모집",사회>교육_시험,"국제사이버대학교,2021학년도,편입생,모집,국제사이버대학교,내년,학년도,편입생,모집...",국제사이버대학교 이버대학교 온라인 직장인 학생들 편입생 대학교 2021학년도 전문가...,국제사이버대학교가 내년 1월7일까지 2021학년도 1학기 신ㆍ편입생을 모집한다고 4...,4
16137,20201203,대전시립미술관 '제2회 대덕에서 과학을 그리다' 수상작품전 아트센터서 전시,IT_과학>과학,"대덕,대전시립미술관,과학,수상,작품전,아트센터,전시,대덕초,이채령,이수,수상,과기부...",대덕 아트센터 한의학 이수인 이채령 과기부 최고상 대전시립미술관 작품전 선승혜 인공...,대전시립미술관은 '제2회 대덕에서 과학을 그리다' 미술대회 수상 작품전을 5일부터 ...,34


<br>
<br>

# **Preprocessing - TRAINING SET**

In [22]:
# 문장 추출
sentences = train['input'].values
sentences[:10]

array(['노무현 비서관 복지부 청와대 내정자 행정학 권덕철 검경 더불어민주당 전해철 여성학 코로나19 목포 잔뼈 전남 한국토지주택공사 한국보건산업진흥원 만큼 산하기관',
       '지지율 한국갤럽 법무부 갤럽 홈페이지 윤석열 임대차 민주당 18세 우상호 더불어민주당 추미애',
       '코로나19 한은 성장률 반도체 건설업 1인 경제성장률 3만 설비투자 한국은행',
       '락토핏 유산균 소비자 종근당 프로바이오틱스 1위 종근당건강 성장세 계열사 건강기능식품 임산부 맘스 국민유산균 관계자',
       '영국 코로나19 존슨 요양원 만큼 노동당 보건장관 버킹엄궁 80세 엘리자베스 90세 스코틀랜드 니컬라 알레그라 화이자 가능성 조너선 행콕 접종이 코로나바이러스 애슈워스',
       '부동산 생활형 평촌 양양 다주택자 숙박시설 1만 취득세 투자처 풀퍼니시드 건축법 양양군 강릉시 냉장고 오션뷰 죽도해변 투기수요 항해수욕장 8억 세탁기',
       '속보치 반도체 3만 설비투자 1인 순수출 gdp 경제통계부장 증가율 국민소득 첨단공정 코로나19 민간소비 음식료품',
       '서동주 애로부부 변호사 미국 방송인 최화정 양재진 법조인 안성맞춤 토크쇼 이상아 홍진경 이용진 출연분 서정희 엔터테이너 서세원 내밀 유명세 맹활약 앞담 제작진 sky 부부 게스트',
       '국제사이버대학교 이버대학교 온라인 직장인 학생들 편입생 대학교 2021학년도 전문가 등록금 교수진 학습자 시니어 이수자 교육과정 교육법 홈페이지 국제사 조상윤 한국인터넷전문가협회 게시판',
       '대덕 아트센터 한의학 이수인 이채령 과기부 최고상 대전시립미술관 작품전 선승혜 인공지능 정부출연연구기관들 과학기술정보통신부 슈퍼컴퓨터 표준과학연구원 157점 대전시립 현미경 미술관장 창의력 원자력 환경보호 과학기술 미래사회'],
      dtype=object)

In [23]:
# 라벨 추출
labels = list(map(str, train['label'].values))
labels[:10]

['9', '18', '47', '50', '11', '31', '24', '0', '4', '34']

In [24]:
train_dataset = pd.DataFrame({'sentence': sentences, 'label': labels}, columns=['sentence', 'label'])

<br>
<br>

# **Preprocessing - TEST SET**

In [25]:
# 문장 추출
sentences = test['input'].values
sentences[:10]

array(['트롯 코치진 트로트 시청자 윤도현 첫방 진시몬 황치열 이상호 마이진 박현호 조정민 조항조 전라 김병현 주영훈 장혜리 남진 이상민 충청 박현빈 윤서령 김범룡 경상 해밍턴 설하윤 조이현 강원 87팀 설운도 응원단장 진해 박구윤 김수희 제주 임하룡 진해성',
       '윤석열 추미애 법무부 행정소송 검찰총장 재판부 대리인 언론사 징계위',
       '장동민 26차례 원주경찰서 원주시 강원 탐문수사 유튜브 국립과학수사원 수사원 가족들 개그맨 승용차 관계자 3개월 2600만 선루프 cctv tv cc 혐의 경찰 피해 구속 차례 범행 주변 남성 사실 조사 설치 결과',
       '서울중앙지방법원 채널a 공판기일 민사소송 방어권 검언 검언유착 진상조사 이동재 해고 소송 부당 조사 채널 혐의 재판 상대 제기 유착 의혹 왜곡 열람 방어 구속 보장 조서 상태 강요 미수 진상 공판 확인 기일 무효 민사 주장 회사 편집',
       '온라인 국립대 코로나19 문재인 사교육 이범 한국 메가스터디 교육부 논술형 사립대 객관식 절대평가 학종 선다형 선다',
       '신세계푸드 안주류 편의점 올반 소비자 리테일 신세계그룹 gs리테일 동파육 hmr 간편식 유통망 100여 닐슨코리아 이마트 이마트24 심플리쿡 계열사 gs25',
       '임실군 유통기한 임실군청 행정처분 신고전화 위생법 식품위생법 감시원 3천만 안전망 소비자들 덮밥류 관리체계 영업자 완제품 과자류 군민들 소비자 다소비식품 다소비 식품위생감시원들 위생관리팀 c마트 b마트 식품 적발 판매 경과 유통 마트',
       '검사장 직무정지 추미애 집단행동 지검장 검찰총장 윤석열 평검사 직무집행 재판부 입장문',
       '노벨상 한국 노벨 수상자 스웨덴 과학자 울렌 대한민국 실시간 이화여대 서울대학교 코로나19 오유경 앨버타대학 연구실 마티아스 마이클 우리나라 인공지능 스크리닝 100만 연구팀',
       '장애인 기아차 코레일 모빌리티 서울 자동차 홈페이지 업무협약 한국철도공사 유류

In [26]:
# 라벨 추출
labels = list(map(str, test['label'].values))
labels[:10]

['2', '20', '7', '20', '4', '50', '24', '7', '32', '29']

In [27]:
test_dataset = pd.DataFrame({'sentence': sentences, 'label': labels}, columns=['sentence', 'label'])

In [28]:
dataset_train = train_dataset.values
dataset_test = test_dataset.values

In [29]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [30]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = []
        self.labels = []
        
        for i in dataset:
            try:
                self.sentences.append(transform([i[sent_idx]]))
                self.labels.append(np.int32(i[label_idx]))
            except Exception as e:
                print(e)
                print(len(self.sentences), len(self.labels))

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [39]:
## Setting parameters
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  2e-5

In [32]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

Argument 'text' has incorrect type (expected str, got float)
645 645
Argument 'text' has incorrect type (expected str, got float)
964 964
Argument 'text' has incorrect type (expected str, got float)
15288 15288
Argument 'text' has incorrect type (expected str, got float)
15814 15814
Argument 'text' has incorrect type (expected str, got float)
28921 28921
Argument 'text' has incorrect type (expected str, got float)
33955 33955
Argument 'text' has incorrect type (expected str, got float)
2239 2239


In [33]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [50]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=NUM_LABELS,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [51]:
model = BERTClassifier(bertmodel,  dr_rate=0.25).to(device)

In [52]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [53]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [54]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [55]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [56]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [57]:
from time import gmtime, strftime
START_TIME = strftime("%Y_%m_%d_%H_%M_%S", gmtime())

In [58]:
SAVE_PATH = BASE_PATH+"/bert_news_model_{}.pth".format(START_TIME)

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    torch.save(model.state_dict(), SAVE_PATH)
    print("Model saved at {}".format(SAVE_PATH))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch 1 batch id 1 loss 4.102231979370117 train acc 0.015625
epoch 1 batch id 201 loss 3.732353925704956 train acc 0.05418221393034826
